In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col, isnan, when, count
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text
import boto3
import ast

In [2]:
SECRET_NAME = "database_tcepb"
REGION_NAME = "sa-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
client = session.client(
    service_name = "secretsmanager",
    region_name = REGION_NAME
)

get_secret_value_response = client.get_secret_value(
    SecretId = SECRET_NAME
)
SERVER_TRIBUNAL = ast.literal_eval(get_secret_value_response["SecretString"])["host"] + ", " + ast.literal_eval(get_secret_value_response["SecretString"])["port"]
DB_TRIBUNAL = ast.literal_eval(get_secret_value_response["SecretString"])["dbname"]
USERNAME = ast.literal_eval(get_secret_value_response["SecretString"])["username"]
PASSWORD = ast.literal_eval(get_secret_value_response["SecretString"])["password"]

string_connection = "DRIVER={SQL Server};" + f"SERVER={SERVER_TRIBUNAL};DATABASE={DB_TRIBUNAL};UID={USERNAME};PWD={PASSWORD}"
url = URL.create("mssql+pyodbc", query={"odbc_connect": string_connection})
engine = create_engine(url) # connection engine

In [3]:
spark = SparkSession.builder \
    .appName("analysing_tables") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.memory", "8G") \
    .config("spark.driver.memory", "45G") \
    .master("local[*]") \
    .getOrCreate() # spark.executor.memory -> ram memory

In [4]:
def read_and_analyse_df(path: str, delimiter: str) -> pyspark.sql.dataframe.DataFrame:
    """Read csv file from path, print Schema, shape and count of Null values per column"""
    df = spark.read.format("csv").options(header="True", delimiter=delimiter, inferSchema="True").load(path).na.drop("all")
                                                                                                            # dropping rows with all values null
    
    df_null_count = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
                            # count when column is NaN or Null
    null_count = df_null_count.collect()[0].asDict()
    columns_to_drop = [c for c, n in null_count.items() if n >= df.count()] # list of all columns with count of null values equal to the number of rows
    df = df.drop(*columns_to_drop) # drop all of those columns
                # starred expression (*variable) will pass the list as arguments

    print("Schema:")
    df.printSchema()
    print(f"Rows: {df.count()}\nColumns: {len(df.columns)}\n")
    print("Null values per column:")
    df_null_count.show()
    
    return df

## Estabelecimento

In [15]:
df_estab_2022_01 = read_and_analyse_df(r"tables\2022\01\tbEstabelecimento202201.csv", delimiter=";")

Schema:
root
 |-- CO_UNIDADE: string (nullable = true)
 |-- CO_CNES: integer (nullable = true)
 |-- NU_CNPJ_MANTENEDORA: long (nullable = true)
 |-- TP_PFPJ: integer (nullable = true)
 |-- NIVEL_DEP: integer (nullable = true)
 |-- NO_RAZAO_SOCIAL: string (nullable = true)
 |-- NO_FANTASIA: string (nullable = true)
 |-- NO_LOGRADOURO: string (nullable = true)
 |-- NU_ENDERECO: string (nullable = true)
 |-- NO_COMPLEMENTO: string (nullable = true)
 |-- NO_BAIRRO: string (nullable = true)
 |-- CO_CEP: integer (nullable = true)
 |-- CO_REGIAO_SAUDE: string (nullable = true)
 |-- CO_MICRO_REGIAO: string (nullable = true)
 |-- CO_DISTRITO_SANITARIO: string (nullable = true)
 |-- CO_DISTRITO_ADMINISTRATIVO: string (nullable = true)
 |-- NU_TELEFONE: string (nullable = true)
 |-- NU_FAX: string (nullable = true)
 |-- NO_EMAIL: string (nullable = true)
 |-- NU_CPF: long (nullable = true)
 |-- NU_CNPJ: long (nullable = true)
 |-- CO_ATIVIDADE: integer (nullable = true)
 |-- CO_CLIENTELA: long (n

In [16]:
df_estab_2022_01.show()

+--------------------+-------+-------------------+-------+---------+--------------------+--------------------+--------------------+-----------+--------------+-----------------+--------+---------------+---------------+---------------------+--------------------------+------------+------------+--------------------+------+--------------+------------+------------+---------+--------------------+------------------+---------------+-----------+----------+--------------------+----------------+-------------------+------------------------------------+------------+----------------+--------------+-------------------+---------------+------+-----------+------------+--------------------------------+--------------+---------------+----------------------+---------------------------+-------------------+-----------------+---------+-------------------------------------------+-----------------------+----------------------+-----------------------+
|          CO_UNIDADE|CO_CNES|NU_CNPJ_MANTENEDORA|TP_PFPJ|NIVEL

### Tipo Estabelecimento

In [17]:
df_tipo_estab_2022_01 = read_and_analyse_df(r"tables\2022\01\tbTipoEstabelecimento202201.csv", delimiter=";")

Schema:
root
 |-- CO_TIPO_ESTABELECIMENTO: integer (nullable = true)
 |-- DS_TIPO_ESTABELECIMENTO: string (nullable = true)

Rows: 26
Columns: 2

Null values per column:
+-----------------------+-----------------------+----------------+
|CO_TIPO_ESTABELECIMENTO|DS_TIPO_ESTABELECIMENTO|DS_CONCEITO_TIPO|
+-----------------------+-----------------------+----------------+
|                      0|                      0|              26|
+-----------------------+-----------------------+----------------+



In [18]:
df_tipo_estab_2022_01.show()

+-----------------------+-----------------------+
|CO_TIPO_ESTABELECIMENTO|DS_TIPO_ESTABELECIMENTO|
+-----------------------+-----------------------+
|                      0|                 OUTROS|
|                      1|   UNIDADE BASICA DE...|
|                      2|   CENTRAL DE GESTAO...|
|                      3|   CENTRAL DE REGULACAO|
|                      4|   CENTRAL DE ABASTE...|
|                      5|   CENTRAL DE TRANSP...|
|                      6|               HOSPITAL|
|                      7|   CENTRO DE ASSISTE...|
|                      8|     PRONTO ATENDIMENTO|
|                      9|               FARMACIA|
|                     10|   UNIDADE DE ATENCA...|
|                     11|   NUCLEO DE TELESSAUDE|
|                     12|   UNIDADE DE ATENCA...|
|                     13|   POLO DE PREVENCAO...|
|                     14|   CASAS DE APOIO A ...|
|                     15|   UNIDADE DE REABIL...|
|                     16|            AMBULATORIO|


### Tipo Unidade

In [19]:
df_tipo_unidade_2022_01 = read_and_analyse_df(r"tables\2022\01\tbTipoUnidade202201.csv", delimiter=";")

Schema:
root
 |-- CO_TIPO_UNIDADE: integer (nullable = true)
 |-- DS_TIPO_UNIDADE: string (nullable = true)

Rows: 39
Columns: 2

Null values per column:
+---------------+---------------+
|CO_TIPO_UNIDADE|DS_TIPO_UNIDADE|
+---------------+---------------+
|              0|              0|
+---------------+---------------+



In [20]:
df_tipo_unidade_2022_01.show()

+---------------+--------------------+
|CO_TIPO_UNIDADE|     DS_TIPO_UNIDADE|
+---------------+--------------------+
|             69|CENTRO DE ATENCAO...|
|             70|CENTRO DE ATENCAO...|
|             71|CENTRO DE APOIO A...|
|             72|UNIDADE DE ATENCA...|
|              1|      POSTO DE SAUDE|
|              2|CENTRO DE SAUDE/U...|
|              4|         POLICLINICA|
|             22| CONSULTORIO ISOLADO|
|             40|UNIDADE MOVEL TER...|
|             42|UNIDADE MOVEL DE ...|
|             32|UNIDADE MOVEL FLU...|
|             36|CLINICA/CENTRO DE...|
|             64|CENTRAL DE REGULA...|
|             43|            FARMACIA|
|             39|UNIDADE DE APOIO ...|
|             61|CENTRO DE PARTO N...|
|             62|HOSPITAL/DIA - IS...|
|             15|       UNIDADE MISTA|
|             20|PRONTO SOCORRO GERAL|
|             21|PRONTO SOCORRO ES...|
+---------------+--------------------+
only showing top 20 rows



### Natureza Jurídica

In [21]:
df_natureza_jur = read_and_analyse_df(r"tables\2022\01\tbNaturezaJuridica202201.csv", delimiter=";")

Schema:
root
 |-- CO_NATUREZA_JUR: integer (nullable = true)
 |-- DS_NATUREZA_JUR: string (nullable = true)

Rows: 97
Columns: 2

Null values per column:
+---------------+---------------+
|CO_NATUREZA_JUR|DS_NATUREZA_JUR|
+---------------+---------------+
|              0|              0|
+---------------+---------------+



In [22]:
df_natureza_jur.show()

+---------------+--------------------+
|CO_NATUREZA_JUR|     DS_NATUREZA_JUR|
+---------------+--------------------+
|           1228|CONSORCIO PUBLICO...|
|           1236|ESTADO OU DISTRIT...|
|           1244|           MUNICIPIO|
|           1252|FUNDACAO PUBLICA ...|
|           1260|FUNDACAO PUBLICA ...|
|           1279|FUNDACAO PUBLICA ...|
|           3280|COMITE FINANCEIRO...|
|           3298|FRENTE PLEBISCITA...|
|           2305|EMPRESA INDIVIDUA...|
|           2313|EMPRESA INDIVIDUA...|
|           3301|ORGANIZAÇÃO SOCIA...|
|           4120|PRODUTOR RURAL (P...|
|           3271|ORGAO DE DIRECAO ...|
|           3263|ORGAO DE DIRECAO ...|
|           3255|ORGAO DE DIRECAO ...|
|           1015|ORGAO PUBLICO DO ...|
|           1023|ORGAO PUBLICO DO ...|
|           1031|ORGAO PUBLICO DO ...|
|           1040|ORGAO PUBLICO DO ...|
|           1058|ORGAO PUBLICO DO ...|
+---------------+--------------------+
only showing top 20 rows



## Leito

In [23]:
df_leito_2022_01 = read_and_analyse_df(r"tables\2022\01\rlEstabComplementar202201.csv", delimiter=";")

Schema:
root
 |-- CO_UNIDADE: string (nullable = true)
 |-- CO_LEITO: integer (nullable = true)
 |-- CO_TIPO_LEITO: double (nullable = true)
 |-- QT_EXIST: integer (nullable = true)
 |-- QT_CONTR: integer (nullable = true)
 |-- QT_SUS: integer (nullable = true)
 |-- TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY'): string (nullable = true)
 |-- CO_USUARIO: string (nullable = true)
 |-- TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY'): string (nullable = true)

Rows: 59394
Columns: 9

Null values per column:
+----------+--------+-------------+-----------+--------+--------+------+------------------------------------+----------+-------------------------------------------+
|CO_UNIDADE|CO_LEITO|CO_TIPO_LEITO|TP_ALTACOMP|QT_EXIST|QT_CONTR|QT_SUS|TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')|CO_USUARIO|TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')|
+----------+--------+-------------+-----------+--------+--------+------+------------------------------------+----------+-------------------------------------------+
|        

In [24]:
df_leito_2022_01.show()

+-------------+--------+-------------+--------+--------+------+------------------------------------+------------+-------------------------------------------+
|   CO_UNIDADE|CO_LEITO|CO_TIPO_LEITO|QT_EXIST|QT_CONTR|QT_SUS|TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')|  CO_USUARIO|TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')|
+-------------+--------+-------------+--------+--------+------+------------------------------------+------------+-------------------------------------------+
|3107405661943|      11|          1.0|       1|    null|     0|                          15/04/2019|    SAOZINHA|                                 22/07/2012|
|3107405661943|      15|          1.0|       1|    null|     0|                          15/04/2019|    SAOZINHA|                                 22/07/2012|
|3136703126277|       7|          7.0|       2|    null|     0|                          16/08/2019|     CNESSCC|                                 22/07/2012|
|2918409192484|      13|          1.0|       6|    n

### Tipo Leito

In [25]:
df_tipo_leito_2022_01 = read_and_analyse_df(r"tables\2022\01\tbLeito202201.csv", delimiter=";")

Schema:
root
 |-- CO_LEITO: integer (nullable = true)
 |-- DS_LEITO: string (nullable = true)
 |-- TP_LEITO: integer (nullable = true)

Rows: 69
Columns: 3

Null values per column:
+--------+--------+--------+
|CO_LEITO|DS_LEITO|TP_LEITO|
+--------+--------+--------+
|       0|       0|       0|
+--------+--------+--------+



In [26]:
df_tipo_leito_2022_01.show()

+--------+--------------------+--------+
|CO_LEITO|            DS_LEITO|TP_LEITO|
+--------+--------------------+--------+
|       1|  BUCO MAXILO FACIAL|       1|
|       2|         CARDIOLOGIA|       1|
|       3|      CIRURGIA GERAL|       1|
|       4|      ENDOCRINOLOGIA|       1|
|       5|   GASTROENTEROLOGIA|       1|
|       6|         GINECOLOGIA|       1|
|       7|CIRURGICO/DIAGNOS...|       7|
|       8|  NEFROLOGIAUROLOGIA|       1|
|       9|       NEUROCIRURGIA|       1|
|      10|OBSTETRICIA CIRUR...|       4|
|      11|        OFTALMOLOGIA|       1|
|      12|           ONCOLOGIA|       1|
|      13|ORTOPEDIATRAUMATO...|       1|
|      14|OTORRINOLARINGOLOGIA|       1|
|      15|            PLASTICA|       1|
|      16|            TORACICA|       1|
|      31|                AIDS|       2|
|      32|         CARDIOLOGIA|       2|
|      33|       CLINICA GERAL|       2|
|      34|            CRONICOS|       6|
+--------+--------------------+--------+
only showing top

## Município

In [27]:
df_municipio_2022_01 = read_and_analyse_df(r"tables\2022\01\tbMunicipio202201.csv", delimiter=";")

Schema:
root
 |-- CO_MUNICIPIO: integer (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- CO_SIGLA_ESTADO: string (nullable = true)
 |-- TP_CADASTRO: string (nullable = true)
 |-- TP_PACTO: string (nullable = true)
 |-- TP_ENVIA: integer (nullable = true)
 |-- TP_ENVIA_CNES: string (nullable = true)
 |-- TP_CIB_SAS: string (nullable = true)
 |-- TP_PLENO_ORIGEM: string (nullable = true)
 |-- TP_MAC: string (nullable = true)
 |-- NU_POPULACAO: integer (nullable = true)
 |-- NU_DENSIDADE: integer (nullable = true)
 |-- CMTP_INICIO_MAC: integer (nullable = true)

Rows: 5606
Columns: 13

Null values per column:
+------------+------------+---------------+-----------+--------+--------+-------------+----------+---------------+------+------------+------------+---------------+
|CO_MUNICIPIO|NO_MUNICIPIO|CO_SIGLA_ESTADO|TP_CADASTRO|TP_PACTO|TP_ENVIA|TP_ENVIA_CNES|TP_CIB_SAS|TP_PLENO_ORIGEM|TP_MAC|NU_POPULACAO|NU_DENSIDADE|CMTP_INICIO_MAC|
+------------+------------+-------------

In [28]:
df_municipio_2022_01.show()

+------------+--------------------+---------------+-----------+--------+--------+-------------+----------+---------------+------+------------+------------+---------------+
|CO_MUNICIPIO|        NO_MUNICIPIO|CO_SIGLA_ESTADO|TP_CADASTRO|TP_PACTO|TP_ENVIA|TP_ENVIA_CNES|TP_CIB_SAS|TP_PLENO_ORIGEM|TP_MAC|NU_POPULACAO|NU_DENSIDADE|CMTP_INICIO_MAC|
+------------+--------------------+---------------+-----------+--------+--------+-------------+----------+---------------+------+------------+------------+---------------+
|      354105|            PRATANIA|             SP|          S|       S|       1|            S|         S|              N|     S|        null|           1|          32018|
|      354130| PRESIDENTE EPITACIO|             SP|          S|       S|       1|            S|         S|              N|     S|        null|        null|          32018|
|      354140| PRESIDENTE PRUDENTE|             SP|          S|       S|       1|            S|         S|              N|     S|        nul

### DIAGM

In [29]:
df_diagm_dados_mapa_2019 = read_and_analyse_df(r"tables\dados_est_muni_2019\dados_estatisticos_agrupamentos_municipais_2019_dados_mapa.csv", delimiter=";")

Schema:
root
 |-- Mun: string (nullable = true)
 |-- Mesorregi�o: string (nullable = true)
 |-- Microrregi�o: string (nullable = true)
 |-- Div 19/20: string (nullable = true)
 |-- Rel 19/20: string (nullable = true)
 |-- �rea territorial - km� [2017]: string (nullable = true)
 |-- Popula��o estimada - pessoas [2018]: double (nullable = true)
 |-- Densidade demogr�fica - hab/km� [2010]: string (nullable = true)
 |-- Escolariza��o 6 a 14 anos - % [2010]: string (nullable = true)
 |-- IDHM �ndice de desenvolvimento humano municipal [2010]: string (nullable = true)
 |-- Mortalidade infantil - �bitos por mil nascidos vivos [2014]: string (nullable = true)
 |-- Or�ament�ria total - Receita(2018): string (nullable = true)
 |-- FPM - Receita(2018): string (nullable = true)
 |-- RPPS - Receita(2018): string (nullable = true)
 |-- FUNDEB - Receita(2018): string (nullable = true)
 |-- SUS - Receita(2018): string (nullable = true)
 |-- Desp Total - Despesa por fun��o (2018): string (nullable = tr

In [30]:
df_diagm_dados_mapa_2019.show()

+-------------------+-----------------+-------------------+---------+---------+-----------------------------+-----------------------------------+--------------------------------------+------------------------------------+------------------------------------------------------+-----------------------------------------------------------+----------------------------------+-------------------+--------------------+----------------------+-------------------+--------------------------------------+---------------------------------+------------------------------------+----------------------------------------------+------------------------------+----+
|                Mun|      Mesorregi�o|       Microrregi�o|Div 19/20|Rel 19/20|�rea territorial - km� [2017]|Popula��o estimada - pessoas [2018]|Densidade demogr�fica - hab/km� [2010]|Escolariza��o 6 a 14 anos - % [2010]|IDHM �ndice de desenvolvimento humano municipal [2010]|Mortalidade infantil - �bitos por mil nascidos vivos [2014]|Or�ament�ria to

In [31]:
df_diagm_dados_est_ibge_2019 = read_and_analyse_df(r"tables\dados_est_muni_2019\dados_estatisticos_agrupamentos_municipais_2019_dados_est_ibge.csv", delimiter=";")

Schema:
root
 |-- Munic�pio [-]: string (nullable = true)
 |-- C�digo [-]: integer (nullable = true)
 |-- Gent�lico [-]: string (nullable = true)
 |-- Prefeito [2017]: string (nullable = true)
 |-- �rea territorial - km� [2017]: double (nullable = true)
 |-- Popula��o estimada - pessoas [2018]: integer (nullable = true)
 |-- Densidade demogr�fica - hab/km� [2010]: double (nullable = true)
 |-- Escolariza��o 6 a 14 anos - % [2010]: double (nullable = true)
 |-- IDHM �ndice de desenvolvimento humano municipal [2010]: double (nullable = true)
 |-- Mortalidade infantil - �bitos por mil nascidos vivos [2014]: string (nullable = true)
 |-- Receitas realizadas - R$ (�1000) [2017]: string (nullable = true)
 |-- Despesas empenhadas - R$ (�1000) [2017]: string (nullable = true)
 |-- PIB per capita - R$ [2016]: string (nullable = true)

Rows: 223
Columns: 13

Null values per column:
+-------------+----------+-------------+---------------+-----------------------------+-----------------------------

In [32]:
df_diagm_dados_est_ibge_2019.show()

+-------------------+----------+----------------+--------------------+-----------------------------+-----------------------------------+--------------------------------------+------------------------------------+------------------------------------------------------+-----------------------------------------------------------+---------------------------------------+---------------------------------------+--------------------------+
|      Munic�pio [-]|C�digo [-]|   Gent�lico [-]|     Prefeito [2017]|�rea territorial - km� [2017]|Popula��o estimada - pessoas [2018]|Densidade demogr�fica - hab/km� [2010]|Escolariza��o 6 a 14 anos - % [2010]|IDHM �ndice de desenvolvimento humano municipal [2010]|Mortalidade infantil - �bitos por mil nascidos vivos [2014]|Receitas realizadas - R$ (�1000) [2017]|Despesas empenhadas - R$ (�1000) [2017]|PIB per capita - R$ [2016]|
+-------------------+----------+----------------+--------------------+-----------------------------+------------------------------

### CNES UFPB

In [33]:
df_cnes_ufpb_2022 = read_and_analyse_df(r"tables\UFPB_CNES\PB2_CNES_2022.csv", delimiter=",")

Schema:
root
 |-- CODUFMUN: integer (nullable = true)
 |-- COMPETEN: integer (nullable = true)
 |-- NUM_CNES: integer (nullable = true)
 |-- NUM_CNES_PF: integer (nullable = true)
 |-- NUM_CNES_PJ: integer (nullable = true)
 |-- NIV_DEP_IND: integer (nullable = true)
 |-- NIV_DEP_MAN: integer (nullable = true)
 |-- VINC_SUS_1: integer (nullable = true)
 |-- VINC_SUS_0: integer (nullable = true)
 |-- TPGESTAO_M: integer (nullable = true)
 |-- TPGESTAO_E: integer (nullable = true)
 |-- TPGESTAO_D: integer (nullable = true)
 |-- QTLEITP1: integer (nullable = true)
 |-- QTLEITP2: integer (nullable = true)
 |-- QTLEITP3: integer (nullable = true)
 |-- NUM_LEIT_HOSP: integer (nullable = true)
 |-- NUM_URGEMERG: integer (nullable = true)
 |-- NUM_ATENDAMB: integer (nullable = true)
 |-- NUM_CENTR_CIR: integer (nullable = true)
 |-- NUM_CENTR_OBS: integer (nullable = true)
 |-- NUM_CENTR_NEO: integer (nullable = true)
 |-- NUM_ATEND_HOSP: integer (nullable = true)
 |-- NUM_ATEND_PREST: integer

In [34]:
df_cnes_ufpb_2022.show()

+--------+--------+--------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+----------+--------+--------+--------+-------------+------------+------------+-------------+-------------+-------------+--------------+---------------+-------+-----------+-----------+---------+------------+------------+------------+------------+------------+-------------+------------+------------+----------+-----------+---------------+-------------+-------------+--------------+----------+------------+----------+----------+
|CODUFMUN|COMPETEN|NUM_CNES|NUM_CNES_PF|NUM_CNES_PJ|NIV_DEP_IND|NIV_DEP_MAN|VINC_SUS_1|VINC_SUS_0|TPGESTAO_M|TPGESTAO_E|TPGESTAO_D|QTLEITP1|QTLEITP2|QTLEITP3|NUM_LEIT_HOSP|NUM_URGEMERG|NUM_ATENDAMB|NUM_CENTR_CIR|NUM_CENTR_OBS|NUM_CENTR_NEO|NUM_ATEND_HOSP|NUM_ATEND_PREST|NUM_MED|NUM_ENF_SUP|NUM_ENF_TEC|NUM_ODONT|NUM_OUT_PROF|NUM_VINCUL_C|NUM_VINCUL_A|NUM_VINCUL_N|NUM_PROF_SUS|NUM_PROF_NSUS|NUM_LT_EXIST|NUM_LT_CONTR|NUM_LT_SUS|NUM_LT_NSUS|NUM_EQUIP_EX

### Internações Hospitalares

In [35]:
df_internacoes_2022_01 = read_and_analyse_df(r"tables\Atendimentos\Internações_Hospitalares_012022.csv", delimiter=";")

Schema:
root
 |-- Munic�pio: string (nullable = true)
 |-- Interna��es: integer (nullable = true)

Rows: 60
Columns: 2

Null values per column:
+---------+-----------+
|Munic�pio|Interna��es|
+---------+-----------+
|        0|          8|
+---------+-----------+



In [36]:
df_internacoes_2022_01.show()

+--------------------+-----------+
|           Munic�pio|Interna��es|
+--------------------+-----------+
|  250010 �gua Branca|         10|
|  250040 Alagoa Nova|         30|
|        250110 Areia|         23|
|     250130 Aroeiras|          3|
|       250180 Bayeux|         66|
|        250190 Bel�m|          1|
|250240 Bonito de ...|         17|
|    250250 Boqueir�o|         17|
|250280 Brejo do Cruz|         25|
|      250300 Caapor�|        147|
|     250320 Cabedelo|        125|
|   250370 Cajazeiras|        593|
|250400 Campina Gr...|       3979|
|250430 Catol� do ...|         97|
|    250440 Concei��o|         24|
|      250480 Coremas|         36|
|        250510 Cuit�|         43|
|    250600 Esperan�a|         30|
|    250630 Guarabira|          1|
|    250690 Itabaiana|        100|
+--------------------+-----------+
only showing top 20 rows



### Atendimentos Ambulatórios

In [37]:
df_atendimentos_amb_2022_01 = spark.read.format("csv").options(header="True", delimiter=";", inferSchema="True") \
                            .load(r"tables\Atendimentos\Atendimentos_Ambulatoriais_012022.csv")

df_atendimentos_amb_2022_01 = df_atendimentos_amb_2022_01.withColumnRenamed(df_atendimentos_amb_2022_01.columns[1], "qtd_aprovada").na.drop("all")

df_null_count = df_atendimentos_amb_2022_01.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_atendimentos_amb_2022_01.columns])
                        # count when column is NaN or Null
null_count = df_null_count.collect()[0].asDict()
columns_to_drop = [c for c, n in null_count.items() if n >= df_atendimentos_amb_2022_01.count()] # list of all columns with count of null values equal to the number of rows
df_atendimentos_amb_2022_01 = df_atendimentos_amb_2022_01.drop(*columns_to_drop) # drop all of those columns
            # starred expression (*variable) will pass the list as arguments

print("Schema:")
df_atendimentos_amb_2022_01.printSchema()
print(f"Rows: {df_atendimentos_amb_2022_01.count()}\nColumns: {len(df_atendimentos_amb_2022_01.columns)}\n")
print("Null values per column:")
df_null_count.show()

Schema:
root
 |-- Munic�pio: string (nullable = true)
 |-- qtd_aprovada: integer (nullable = true)

Rows: 224
Columns: 2

Null values per column:
+---------+------------+
|Munic�pio|qtd_aprovada|
+---------+------------+
|        0|           8|
+---------+------------+



In [38]:
df_atendimentos_amb_2022_01.show()

+--------------------+------------+
|           Munic�pio|qtd_aprovada|
+--------------------+------------+
|  250010 �gua Branca|        2296|
|250030 Alagoa Grande|        5796|
|  250040 Alagoa Nova|       14850|
|    250050 Alagoinha|        2610|
|     250053 Alcantil|        1086|
|250057 Algod�o de...|         941|
|     250060 Alhandra|        4366|
|       250073 Amparo|         110|
|    250077 Aparecida|         124|
|      250080 Ara�agi|        8846|
|        250090 Arara|        3227|
|      250100 Araruna|        5393|
|        250110 Areia|       20962|
|250115 Areia de B...|          12|
|       250120 Areial|        2128|
|     250130 Aroeiras|       11566|
|     250135 Assun��o|         153|
|250140 Ba�a da Tr...|        3901|
|   250150 Bananeiras|        3189|
|      250153 Bara�na|         667|
+--------------------+------------+
only showing top 20 rows



### Empenhos Saúde

In [5]:
with open(r"tables\Empenhos.sql") as f:
    query_empenhos_2022 = f.read()

with engine.begin() as conn:
    result = conn.execute(text(query_empenhos_2022))
    rows_empenhos_2022 = []
    for r in result:
        rows_empenhos_2022.append(list(r))

for i in range(len(rows_empenhos_2022)):
    rows_empenhos_2022[i][-1] = float(rows_empenhos_2022[i][-1])


In [6]:
schema_empenhos = StructType([
    StructField(name="Mês/Ano", dataType=StringType(), nullable=False),
    StructField(name="cd_ibge", dataType=StringType(), nullable=False),
    StructField(name="no_Municipio", dataType=StringType(), nullable=False),
    StructField(name="vl_empenhos_saude", dataType=FloatType(), nullable=False)
])

In [7]:
df_empenhos_2022 = spark.createDataFrame(data=rows_empenhos_2022, schema=schema_empenhos)
print("Schema:")
df_empenhos_2022.printSchema()

print(f"Rows: {df_empenhos_2022.count()}\nColumns: {len(df_empenhos_2022.columns)}\n")

print("Null values per column:")
df_empenhos_2022.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_empenhos_2022.columns]).show()

Schema:
root
 |-- Mês/Ano: string (nullable = false)
 |-- cd_ibge: string (nullable = false)
 |-- no_Municipio: string (nullable = false)
 |-- vl_empenhos_saude: float (nullable = false)

Rows: 2676
Columns: 4

Null values per column:
+-------+-------+------------+-----------------+
|Mês/Ano|cd_ibge|no_Municipio|vl_empenhos_saude|
+-------+-------+------------+-----------------+
|      0|      0|           0|                0|
+-------+-------+------------+-----------------+



In [8]:
df_empenhos_2022.show()

+-------+-------+------------+-----------------+
|Mês/Ano|cd_ibge|no_Municipio|vl_empenhos_saude|
+-------+-------+------------+-----------------+
|01/2022| 250010| Agua Branca|        421441.12|
|02/2022| 250010| Agua Branca|        778324.94|
|03/2022| 250010| Agua Branca|        1129195.0|
|04/2022| 250010| Agua Branca|        683457.56|
|05/2022| 250010| Agua Branca|        1093227.4|
|06/2022| 250010| Agua Branca|        923915.44|
|07/2022| 250010| Agua Branca|         956457.2|
|08/2022| 250010| Agua Branca|        1010722.6|
|09/2022| 250010| Agua Branca|        908292.56|
|10/2022| 250010| Agua Branca|         867565.5|
|11/2022| 250010| Agua Branca|        1042141.3|
|12/2022| 250010| Agua Branca|         909351.2|
|01/2022| 250020|      Aguiar|        267563.25|
|02/2022| 250020|      Aguiar|         595630.8|
|03/2022| 250020|      Aguiar|         634644.8|
|04/2022| 250020|      Aguiar|        526592.44|
|05/2022| 250020|      Aguiar|        646825.75|
|06/2022| 250020|   

## Atendimento_Convênio

In [9]:
df_atend_conv_2022_01 = read_and_analyse_df(r"tables\2022\01\rlEstabAtendPrestConv202201.csv", delimiter=";")

Schema:
root
 |-- CO_UNIDADE: string (nullable = true)
 |-- CO_ATENDIMENTO_PRESTADO: integer (nullable = true)
 |-- CO_CONVENIO: integer (nullable = true)
 |-- CO_USUARIO: string (nullable = true)
 |-- TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY'): string (nullable = true)

Rows: 815917
Columns: 5

Null values per column:
+----------+-----------------------+-----------+----------+------------------------------------+-------------------------------------------+
|CO_UNIDADE|CO_ATENDIMENTO_PRESTADO|CO_CONVENIO|CO_USUARIO|TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')|TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')|
+----------+-----------------------+-----------+----------+------------------------------------+-------------------------------------------+
|         0|                      0|          0|      5002|                                 143|                                     815917|
+----------+-----------------------+-----------+----------+------------------------------------+-------------------------

In [10]:
df_atend_conv_2022_01.show()

+-------------+-----------------------+-----------+-----------+------------------------------------+
|   CO_UNIDADE|CO_ATENDIMENTO_PRESTADO|CO_CONVENIO| CO_USUARIO|TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')|
+-------------+-----------------------+-----------+-----------+------------------------------------+
|2501900621986|                      3|          1|      SCNES|                          22/03/2021|
|4202700613495|                      2|          1|      SAUDE|                          19/04/2021|
|4202700613495|                      3|          1|      SAUDE|                          19/04/2021|
|4202700613495|                      4|          1|      SAUDE|                          19/04/2021|
|4202700613495|                      6|          1|      SAUDE|                          19/04/2021|
|4305200650641|                      2|          1|     DANIEL|                          29/03/2021|
|4309109847294|                      2|          1|      SAUDE|                          22

### Atendimento

In [11]:
df_atend_prestado_2022_01 = read_and_analyse_df(r"tables\2022\01\tbAtendimentoPrestado202201.csv", delimiter=";")

Schema:
root
 |-- CO_ATENDIMENTO_PRESTADO: integer (nullable = true)
 |-- DS_ATENDIMENTO_PRESTADO: string (nullable = true)

Rows: 7
Columns: 2

Null values per column:
+-----------------------+-----------------------+
|CO_ATENDIMENTO_PRESTADO|DS_ATENDIMENTO_PRESTADO|
+-----------------------+-----------------------+
|                      0|                      0|
+-----------------------+-----------------------+



In [12]:
df_atend_prestado_2022_01.show()

+-----------------------+-----------------------+
|CO_ATENDIMENTO_PRESTADO|DS_ATENDIMENTO_PRESTADO|
+-----------------------+-----------------------+
|                      1|             INTERNACAO|
|                      2|           AMBULATORIAL|
|                      3|                   SADT|
|                      4|               URGENCIA|
|                      5|                 OUTROS|
|                      6|    VIGILANCIA EM SAUDE|
|                      7|              REGULACAO|
+-----------------------+-----------------------+



### Convênio

In [13]:
df_convenio_2022_01 = read_and_analyse_df(r"tables\2022\01\tbConvenio202201.csv", delimiter=";")

Schema:
root
 |-- CO_CONVENIO: integer (nullable = true)
 |-- DS_CONVENIO: string (nullable = true)

Rows: 7
Columns: 2

Null values per column:
+-----------+-----------+
|CO_CONVENIO|DS_CONVENIO|
+-----------+-----------+
|          0|          0|
+-----------+-----------+



In [14]:
df_convenio_2022_01.show()

+-----------+--------------------+
|CO_CONVENIO|         DS_CONVENIO|
+-----------+--------------------+
|          1|                 SUS|
|          2|          PARTICULAR|
|          3|PLANO / SEGURO PR...|
|          4|PLANO / SEGURO TE...|
|          5|PLANO DE SAUDE PU...|
|          6|PLANO DE SAUDE PR...|
|          7|          GRATUIDADE|
+-----------+--------------------+



## Carga Horária SUS

In [5]:
df_carga_hor_2022_01 = spark.read.format("csv").options(header="True", delimiter=";", inferSchema="True").load(r"tables\2022\01\tbCargaHorariaSus202201.csv")

df_carga_hor_2022_01 = df_carga_hor_2022_01.withColumnRenamed(df_carga_hor_2022_01.columns[-5], "DT_ATUALIZACAO") \
                                        .withColumnRenamed(df_carga_hor_2022_01.columns[-3], "DT_ATUALIZACAO_ORIGEM") \
                                        .na.drop("all")
                                        # dropping rows with all values null

df_null_count = df_carga_hor_2022_01.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_carga_hor_2022_01.columns])
                        # count when column is NaN or Null
null_count = df_null_count.collect()[0].asDict()
columns_to_drop = [c for c, n in null_count.items() if n >= df_carga_hor_2022_01.count()] # list of all columns with count of null values equal to the number of rows
df_carga_hor_2022_01 = df_carga_hor_2022_01.drop(*columns_to_drop) # drop all of those columns
            # starred expression (*variable) will pass the list as arguments

print("Schema:")
df_carga_hor_2022_01.printSchema()
print(f"Rows: {df_carga_hor_2022_01.count()}\nColumns: {len(df_carga_hor_2022_01.columns)}\n")
print("Null values per column:")
df_null_count.show()

Schema:
root
 |-- CO_UNIDADE: string (nullable = true)
 |-- CO_PROFISSIONAL_SUS: string (nullable = true)
 |-- CO_CBO: string (nullable = true)
 |-- TP_SUS_NAO_SUS: string (nullable = true)
 |-- IND_VINCULACAO: integer (nullable = true)
 |-- TP_TERCEIRO_SIH: string (nullable = true)
 |-- QT_CARGA_HORARIA_AMBULATORIAL: integer (nullable = true)
 |-- CO_CONSELHO_CLASSE: string (nullable = true)
 |-- NU_REGISTRO: string (nullable = true)
 |-- SG_UF_CRM: string (nullable = true)
 |-- TP_PRECEPTOR: integer (nullable = true)
 |-- TP_RESIDENTE: integer (nullable = true)
 |-- NU_CNPJ_DETALHAMENTO_VINCULO: string (nullable = true)
 |-- DT_ATUALIZACAO: string (nullable = true)
 |-- CO_USUARIO: string (nullable = true)
 |-- QT_CARGA_HORARIA_OUTROS: integer (nullable = true)
 |-- QT_CARGA_HOR_HOSP_SUS: integer (nullable = true)

Rows: 5094745
Columns: 17

Null values per column:
+----------+-------------------+------+--------------+--------------+---------------+-----------------------------+-----

In [6]:
df_carga_hor_2022_01.show()

+--------------------+-------------------+------+--------------+--------------+---------------+-----------------------------+------------------+-----------+---------+------------+------------+----------------------------+--------------+-----------+-----------------------+---------------------+
|          CO_UNIDADE|CO_PROFISSIONAL_SUS|CO_CBO|TP_SUS_NAO_SUS|IND_VINCULACAO|TP_TERCEIRO_SIH|QT_CARGA_HORARIA_AMBULATORIAL|CO_CONSELHO_CLASSE|NU_REGISTRO|SG_UF_CRM|TP_PRECEPTOR|TP_RESIDENTE|NU_CNPJ_DETALHAMENTO_VINCULO|DT_ATUALIZACAO| CO_USUARIO|QT_CARGA_HORARIA_OUTROS|QT_CARGA_HOR_HOSP_SUS|
+--------------------+-------------------+------+--------------+--------------+---------------+-----------------------------+------------------+-----------+---------+------------+------------+----------------------------+--------------+-----------+-----------------------+---------------------+
|AP000000000000000...|   1EFFD0E1186062EE|515105|             S|         10000|              2|                    

### Profissional SUS

In [7]:
df_profissional_2022_01 = read_and_analyse_df(r"tables\2022\01\tbDadosProfissionalSus202201.csv", delimiter=";")

Schema:
root
 |-- CO_PROFISSIONAL_SUS: string (nullable = true)
 |-- 'CO_CPF': string (nullable = true)
 |-- NO_PROFISSIONAL: string (nullable = true)
 |-- CO_CNS: long (nullable = true)
 |-- TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY'): string (nullable = true)
 |-- CO_USUARIO: string (nullable = true)
 |-- ST_NMPROF_CADSUS: string (nullable = true)
 |-- CO_NACIONALIDADE: integer (nullable = true)
 |-- CO_SEQ_INCLUSAO: integer (nullable = true)
 |-- TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY'): string (nullable = true)

Rows: 5725003
Columns: 10

Null values per column:
+-------------------+--------+---------------+------+------------------------------------+----------+----------------+----------------+---------------+-------------------------------------------+
|CO_PROFISSIONAL_SUS|'CO_CPF'|NO_PROFISSIONAL|CO_CNS|TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')|CO_USUARIO|ST_NMPROF_CADSUS|CO_NACIONALIDADE|CO_SEQ_INCLUSAO|TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')|
+-------------------+--------+---------

In [8]:
df_profissional_2022_01.show()

+-------------------+--------+--------------------+---------------+------------------------------------+-----------+----------------+----------------+---------------+-------------------------------------------+
|CO_PROFISSIONAL_SUS|'CO_CPF'|     NO_PROFISSIONAL|         CO_CNS|TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')| CO_USUARIO|ST_NMPROF_CADSUS|CO_NACIONALIDADE|CO_SEQ_INCLUSAO|TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')|
+-------------------+--------+--------------------+---------------+------------------------------------+-----------+----------------+----------------+---------------+-------------------------------------------+
|   7F388F50D27949AB|  CO_CPF|EDUARDO ROSA DE O...|706301181314780|                          28/07/2021|     HDLAPA|            null|            null|           null|                                       null|
|   FFC20DB5BED8BE78|  CO_CPF|KAROLINE APARECID...|705006482591359|                          17/03/2020|      BIGUA|            null|            null|      

### Conselho Classe

In [9]:
df_cons_classe_2022_01 = read_and_analyse_df(r"tables\2022\01\tbConselhoClasse202201.csv", delimiter=";")

Schema:
root
 |-- CO_CONSELHO_CLASSE: integer (nullable = true)
 |-- DS_CONSELHO_CLASSE: string (nullable = true)

Rows: 12
Columns: 2

Null values per column:
+------------------+------------------+
|CO_CONSELHO_CLASSE|DS_CONSELHO_CLASSE|
+------------------+------------------+
|                 0|                 0|
+------------------+------------------+



In [10]:
df_cons_classe_2022_01.show()

+------------------+--------------------+
|CO_CONSELHO_CLASSE|  DS_CONSELHO_CLASSE|
+------------------+--------------------+
|                 1|    CR ADMINISTRACAO|
|                 2|CR ASSISTENTE SOCIAL|
|                 3|  CR BIBLIOTECONOMIA|
|                 4|    CR CONTABILIDADE|
|                 5|       CR ENFERMAGEM|
|                 6|CR ENGENHARIA, AG...|
|                 7|      CR ESTATISTICA|
|                 8|         CR FARMACIA|
|                 9|CR FISIOTERAPIA E...|
|                10|         CR MEDICINA|
|                11|       CR PSICOLOGIA|
|                12|                 OAB|
+------------------+--------------------+



### Atividade Profissional

In [11]:
df_ativ_prof_2022_01 = read_and_analyse_df(r"tables\2022\01\tbAtividadeProfissional202201.csv", delimiter=";")

Schema:
root
 |-- CO_CBO: string (nullable = true)
 |-- DS_ATIVIDADE_PROFISSIONAL: string (nullable = true)
 |-- TP_CBO_SAUDE: string (nullable = true)
 |-- ST_CBO_REGULAMENTADO: string (nullable = true)

Rows: 2620
Columns: 4

Null values per column:
+------+-------------------------+-----------------------------+------------+--------------------+-----------+
|CO_CBO|DS_ATIVIDADE_PROFISSIONAL|TP_CLASSIFICACAO_PROFISSIONAL|TP_CBO_SAUDE|ST_CBO_REGULAMENTADO|NO_ANO_CMPT|
+------+-------------------------+-----------------------------+------------+--------------------+-----------+
|     0|                        0|                         2620|           0|                   9|       2620|
+------+-------------------------+-----------------------------+------------+--------------------+-----------+



In [12]:
df_ativ_prof_2022_01.show()

+------+-------------------------+------------+--------------------+
|CO_CBO|DS_ATIVIDADE_PROFISSIONAL|TP_CBO_SAUDE|ST_CBO_REGULAMENTADO|
+------+-------------------------+------------+--------------------+
|513435|     ATENDENTE DE LANC...|           N|                   N|
|513440|                  BARISTA|           N|                   N|
|513505|     AUXILIAR NOS SERV...|           N|                   N|
|513605|            CHURRASQUEIRO|           N|                   N|
|513610|                PIZZAIOLO|           N|                   N|
|513615|                 SUSHIMAN|           N|                   N|
|514105|             ASCENSORISTA|           N|                   N|
|514110|                GARAGISTA|           N|                   N|
|514115|                SACRISTAO|           N|                   N|
|514120|      ZELADOR DE EDIFICIO|           N|                   N|
|514205|     COLETOR DE LIXO D...|           N|                   N|
|514215|          VARREDOR DE RUA|

## Equipe Profissionais

In [13]:
df_equipe_2022_01 = read_and_analyse_df(r"tables\2022\01\tbEquipe202201.csv", delimiter=";")

Schema:
root
 |-- CO_MUNICIPIO: integer (nullable = true)
 |-- CO_AREA: integer (nullable = true)
 |-- SEQ_EQUIPE: integer (nullable = true)
 |-- CO_UNIDADE: long (nullable = true)
 |-- TP_EQUIPE: integer (nullable = true)
 |-- CO_SUB_TIPO_EQUIPE: integer (nullable = true)
 |-- NO_REFERENCIA: string (nullable = true)
 |-- DT_ATIVACAO: string (nullable = true)
 |-- DT_DESATIVACAO: string (nullable = true)
 |-- TP_POP_ASSIST_QUILOMB: integer (nullable = true)
 |-- TP_POP_ASSIST_ASSENT: integer (nullable = true)
 |-- TP_POP_ASSIST_GERAL: integer (nullable = true)
 |-- TP_POP_ASSIST_ESCOLA: integer (nullable = true)
 |-- TP_POP_ASSIST_PRONASCI: integer (nullable = true)
 |-- TP_POP_ASSIST_INDIGENA: integer (nullable = true)
 |-- TP_POP_ASSIST_RIBEIRINHA: integer (nullable = true)
 |-- TP_POP_ASSIST_SITUACAO_RUA: integer (nullable = true)
 |-- TP_POP_ASSIST_PRIV_LIBERDADE: integer (nullable = true)
 |-- TP_POP_ASSIST_CONFLITO_LEI: integer (nullable = true)
 |-- TP_POP_ASSIST_ADOL_CONF_LEI: 

In [14]:
df_equipe_2022_01.show()

+------------+-------+----------+-------------+---------+------------------+--------------------+-----------+--------------+---------------------+--------------------+-------------------+--------------------+----------------------+----------------------+------------------------+--------------------------+----------------------------+--------------------------+---------------------------+-----------+-------------+-----------------+-------------+---------+------------------------------------+----------+-------------------------------------------+
|CO_MUNICIPIO|CO_AREA|SEQ_EQUIPE|   CO_UNIDADE|TP_EQUIPE|CO_SUB_TIPO_EQUIPE|       NO_REFERENCIA|DT_ATIVACAO|DT_DESATIVACAO|TP_POP_ASSIST_QUILOMB|TP_POP_ASSIST_ASSENT|TP_POP_ASSIST_GERAL|TP_POP_ASSIST_ESCOLA|TP_POP_ASSIST_PRONASCI|TP_POP_ASSIST_INDIGENA|TP_POP_ASSIST_RIBEIRINHA|TP_POP_ASSIST_SITUACAO_RUA|TP_POP_ASSIST_PRIV_LIBERDADE|TP_POP_ASSIST_CONFLITO_LEI|TP_POP_ASSIST_ADOL_CONF_LEI|CO_CNES_UOM|NU_CH_AMB_UOM|CD_MOTIVO_DESATIV|CD_TP_DESATIV|C

### Tipo Equipe

In [15]:
df_tipo_equipe_2022_01 = read_and_analyse_df(r"tables\2022\01\tbTipoEquipe202201.csv", delimiter=";")

Schema:
root
 |-- TP_EQUIPE: integer (nullable = true)
 |-- DS_EQUIPE: string (nullable = true)
 |-- CO_GRUPO_EQUIPE: integer (nullable = true)

Rows: 64
Columns: 3

Null values per column:
+---------+---------+---------------+
|TP_EQUIPE|DS_EQUIPE|CO_GRUPO_EQUIPE|
+---------+---------+---------------+
|        0|        0|              0|
+---------+---------+---------------+



In [16]:
df_tipo_equipe_2022_01.show()

+---------+--------------------+---------------+
|TP_EQUIPE|           DS_EQUIPE|CO_GRUPO_EQUIPE|
+---------+--------------------+---------------+
|       48|EESE - EQUIPE EXC...|             13|
|       50|EABP1   - EQ ATEN...|             14|
|       51|EABP1SM - EQ ATEN...|             14|
|       52|EABP2   - EQ ATEN...|             14|
|       53|EABP2SM - EQ ATEN...|             14|
|       54|EABP3   - EQ ATEN...|             14|
|       46|EMAD T2 - EQUIPE ...|             17|
|       47|ECD - EQUIPE DE C...|             19|
|       49|                 EAP|             12|
|       58|eMAESM1 - EQ. MUL...|             15|
|       59|eMAESM2 - EQ. MUL...|             15|
|       60|eMAESM3 - EQ. MUL...|             15|
|       70|ESF - EQUIPE DE S...|             20|
|       71|ESB - EQUIPE DE S...|             21|
|       72|ENASFAP - EQ DO N...|             22|
|       73|ECR - EQUIPE DOS ...|             23|
|       74|EAPP - EQUIPE DE ...|             24|
|       75|EMAESM - 

### Grupo Equipe

In [17]:
df_grupo_equipe_2022_01 = read_and_analyse_df(r"tables\2022\01\tbGrupoEquipe202201.csv", delimiter=";")

Schema:
root
 |-- CO_GRUPO_EQUIPE: integer (nullable = true)
 |-- NO_GRUPO_EQUIPE: string (nullable = true)

Rows: 26
Columns: 2

Null values per column:
+---------------+---------------+
|CO_GRUPO_EQUIPE|NO_GRUPO_EQUIPE|
+---------------+---------------+
|              0|              0|
+---------------+---------------+



In [18]:
df_grupo_equipe_2022_01.show()

+---------------+---------------+
|CO_GRUPO_EQUIPE|NO_GRUPO_EQUIPE|
+---------------+---------------+
|             14|           EABP|
|             12|            EAP|
|             13|           EESE|
|              1|            ESF|
|              2|           EACS|
|              3|           EPEN|
|              4|           NASF|
|              5|           EMSI|
|              6|          ESFPR|
|              7|           ESFF|
|              8|            EAB|
|              9|            EAD|
|             10|            ECR|
|             11|            ESB|
|             15|         EMAESM|
|             16|          EMSIN|
|             17|          EMADN|
|             18|          EMAPN|
|             19|           ECDN|
|             20|           ESFN|
+---------------+---------------+
only showing top 20 rows



## Atividade

In [19]:
df_atividade_2022_01 = read_and_analyse_df(r"tables\2022\01\tbAtividade202201.csv", delimiter=";")

Schema:
root
 |-- CO_GRUPO_ATIVIDADE: integer (nullable = true)
 |-- CO_ATIVIDADE: integer (nullable = true)
 |-- DS_ATIVIDADE: string (nullable = true)
 |-- DS_CONCEITO_ATIVIDADE: string (nullable = true)

Rows: 28
Columns: 4

Null values per column:
+------------------+------------+------------+---------------------+
|CO_GRUPO_ATIVIDADE|CO_ATIVIDADE|DS_ATIVIDADE|DS_CONCEITO_ATIVIDADE|
+------------------+------------+------------+---------------------+
|                 0|           0|           0|                    0|
+------------------+------------+------------+---------------------+



In [20]:
df_atividade_2022_01.show()

+------------------+------------+--------------------+---------------------+
|CO_GRUPO_ATIVIDADE|CO_ATIVIDADE|        DS_ATIVIDADE|DS_CONCEITO_ATIVIDADE|
+------------------+------------+--------------------+---------------------+
|                 2|          19|ANALISES LABORATO...| Ações e serviços ...|
|                 2|          20|VIGILANCIA EM SAU...| Ações e serviços ...|
|                 2|          21|ESCLARECIMENTO DA...| Ações e serviços ...|
|                 3|          22|       ADMINISTRACAO| Compreende os pro...|
|                 3|          23|REGULACAO ASSISTE...| Compreende a orga...|
|                 3|          24|LOGISTICA DE INSUMOS| Compreende o arma...|
|                 3|          25|LOGISTICA DE ORGA...| Compreende as açõ...|
|                 4|          26|       HOSPITALIDADE| Serviços que visa...|
|                 4|          27|CONFECCAO DE ORTE...| Serviço de fabric...|
|                 0|           0|       NAO SE APLICA|         Sem Conceito|

### Grupo Atividade

In [21]:
df_grupo_atividade_2022_01 = read_and_analyse_df(r"tables\2022\01\tbGrupoAtividade202201.csv", delimiter=";")

Schema:
root
 |-- CO_GRUPO_ATIVIDADE: integer (nullable = true)
 |-- DS_GRUPO_ATIVIDADE: string (nullable = true)

Rows: 5
Columns: 2

Null values per column:
+------------------+------------------+
|CO_GRUPO_ATIVIDADE|DS_GRUPO_ATIVIDADE|
+------------------+------------------+
|                 0|                 0|
+------------------+------------------+



In [22]:
df_grupo_atividade_2022_01.show()

+------------------+--------------------+
|CO_GRUPO_ATIVIDADE|  DS_GRUPO_ATIVIDADE|
+------------------+--------------------+
|                 0|       NAO SE APLICA|
|                 1| ASSISTENCIA A SAUDE|
|                 2| VIGILANCIA EM SAUDE|
|                 3|     GESTAO DA SAUDE|
|                 4|OUTRAS ATIVIDADES...|
+------------------+--------------------+

